In [82]:
import requests
from bs4 import BeautifulSoup
import re
import sys
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk import stem
import nltk
nltk.download('wordnet')
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
import time

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\49178\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [83]:
def gets_links(artist:str)->list:
    base_url= "https://www.azlyrics.com"
    #url = f"{ base_url}/{artist[0]}/{artist}.html"
    url = base_url + "/{}/{}.html".format(artist[0],artist)
 # param_dict = {'db':'pubmed', 'term':'escherichia', 'rettype':'uilist'}
 # headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    linkarray = []
    for link in soup.find_all(href=re.compile(f"/lyrics/{artist}")):
        href = link.get('href')
        removedots = href[2:]
        linkarray.append(base_url + removedots)
    return linkarray
    time.sleep(10)

In [84]:
art1=gets_links('kaleo')   #getting links
art2=gets_links('logic')

In [85]:
def songs_lyrics():   #selecting the songs 
    time.sleep(10)
    urls={}
    urls['nogood']=art1[10]
    urls['vor']=art1[9]
    urls['glasshouse']=art1[0]
    urls['fool']=art1[3]
    urls['broken']=art1[2]
    urls['hunger']=art2[2]
    urls['times']=art2[9]
    urls['logic']=art2[1]
    urls['tip']=art2[4]
    urls['onlock']= art2[5]
    urls['background']=art2[3]
    urls['stain']=art2[12]
    return urls
    
    


In [86]:
urls=songs_lyrics() #defining the variable with the song_lyrics list
urls

{'nogood': 'https://www.azlyrics.com/lyrics/kaleo/nogood.html',
 'vor': 'https://www.azlyrics.com/lyrics/kaleo/vorvaglaskgi.html',
 'glasshouse': 'https://www.azlyrics.com/lyrics/kaleo/glasshouse.html',
 'fool': 'https://www.azlyrics.com/lyrics/kaleo/fool.html',
 'broken': 'https://www.azlyrics.com/lyrics/kaleo/brokenbones.html',
 'hunger': 'https://www.azlyrics.com/lyrics/logic/hunger.html',
 'times': 'https://www.azlyrics.com/lyrics/logic/timeigive.html',
 'logic': 'https://www.azlyrics.com/lyrics/logic/logiconthebeat.html',
 'tip': 'https://www.azlyrics.com/lyrics/logic/95tip.html',
 'onlock': 'https://www.azlyrics.com/lyrics/logic/onlock.html',
 'background': 'https://www.azlyrics.com/lyrics/logic/backgroundcheck.html',
 'stain': 'https://www.azlyrics.com/lyrics/logic/staininthegame.html'}

In [87]:
artists=['kaleo']*5+['logic']*7    #assigning the singers to the songs
artists=pd.DataFrame(artists) ##created a dataframe with the list artist


In [88]:
lyrics_data=[]    #assigning the singers names to the corresponding songs' lyrics
for lyric in urls:
    url=urls[lyric]
    print(lyric, url)
    time.sleep(10)
    request=requests.get(url).text
    soup=BeautifulSoup(request)
    lyrics=[x.text for x in soup.find_all(attrs={'class':"col-xs-12 col-lg-8 text-center"})][0]
    lyrics = lyrics.replace("\n"," ")
    lyrics = lyrics.split("Submit Corrections",1)[0]
    lyrics = lyrics.split("Lyrics",1)[1]
    lyrics_data.append(lyrics)
    
df=pd.DataFrame(lyrics_data) #creating a dataframe with the list of lyrics
df.to_csv('lyrics_data.csv') #convert and save it csv file
print(df)
   


nogood https://www.azlyrics.com/lyrics/kaleo/nogood.html
vor https://www.azlyrics.com/lyrics/kaleo/vorvaglaskgi.html
glasshouse https://www.azlyrics.com/lyrics/kaleo/glasshouse.html
fool https://www.azlyrics.com/lyrics/kaleo/fool.html
broken https://www.azlyrics.com/lyrics/kaleo/brokenbones.html
hunger https://www.azlyrics.com/lyrics/logic/hunger.html
times https://www.azlyrics.com/lyrics/logic/timeigive.html
logic https://www.azlyrics.com/lyrics/logic/logiconthebeat.html
tip https://www.azlyrics.com/lyrics/logic/95tip.html
onlock https://www.azlyrics.com/lyrics/logic/onlock.html
background https://www.azlyrics.com/lyrics/logic/backgroundcheck.html
stain https://www.azlyrics.com/lyrics/logic/staininthegame.html
                                                    0
0        "No Good"   \r Can't fight the temptation...
1        "Vor Ã VaglaskÃ³gi"   \r KvÃ¶ldiÃ° er ok...
2        "Glass House"   \r Well I don't need much...
3        "Fool"   \r Oh yeah, come on That's right...
4        

In [89]:
df['artists']=artists   #defining the column name of artist
df.columns =['lyrics','artists'] #defining all the columns
df.to_csv('lyrics_data.csv') #save to csv file
df

,lyrics,artists
0,"""No Good"" \r Can't fight the temptation...",kaleo
1,"""Vor Ã VaglaskÃ³gi"" \r KvÃ¶ldiÃ° er ok...",kaleo
2,"""Glass House"" \r Well I don't need much...",kaleo
3,"""Fool"" \r Oh yeah, come on That's right...",kaleo
4,"""Broken Bones"" \r The devil's going to ...",kaleo
5,"""Hunger"" \r Psychological! Damn right H...",logic
6,"""Time I Give"" \r I know the time I give...",logic
7,"""Logic On The Beat"" \r Yeah it's Logic ...",logic
8,"""'95 Tip"" \r This right here, homie Thi...",logic
9,"""On Lock"" \r Going so you know I'm tryi...",logic


In [90]:
X=df['lyrics'] #defining x
y=df['artists'] #defininig y

In [91]:
print(X[0]) #checking how does X look

 Can't fight the temptation When you get the vibration Won't do you no good Won't do you no good  You better start running When you hear the man coming Won't do you no good Won't do you no good  No we don't mind If you don't mind Hell I never mind  Don't let the mold break you Let the feeling come take you Won't do you no good Won't do you no good  And you know I got fever When you hit me right, you know you might as well Won't do you no good Said it won't do you no good  And I said, kiss your baby goodbye Come on, love, it's alright! Heaven knows they wanna break you apart, yeah  Kiss your baby goodbye Come on, love, it's alright! You never know unless you give it a try Oh baby  Said do you no good  Can't fight the temptation When you get the vibration Won't do you no good Won't do you no good And you can tell I got fever You try to shake it off, try it all you want Won't do you no good Said it won't do you no good  And I said, kiss your baby goodbye Come on, love, it's alright Heaven

In [92]:
len(X[0]) #checking the length of the first lyrics in X

1346

In [93]:
import spacy 
model = spacy.load('en_core_web_md')

In [94]:
def clean_my_string(string):
    """use spacy to tokenize, lemmatize and remove stop words"""
    clean_string = []
    token_string = model(string)
    for token in token_string:
        if not token.is_stop and not token.is_punct and not token.like_num:
            clean_string.append(token.lemma_)
    return ' '.join(clean_string)

In [95]:
clean_lyrics_data=[]
for item in X: 
    item=clean_my_string(item)
    clean_lyrics_data.append(item)

In [96]:
#checking the difference in length after spacy cleaning
print(len(X[1]),len(clean_lyrics_data[1]))


670 703


In [97]:
#saving to pickles
with open('clean_lyrics_data.pickle', 'wb') as file:
    pickle.dump(clean_lyrics_data,file)

In [98]:
#loading pickled files
with open('clean_lyrics_data.pickle', 'rb') as f:
      clean_lyrics_data = pickle.load(f)

In [99]:
df1=pd.DataFrame(clean_lyrics_data) 
df1.columns =['lyrics']

In [100]:
df['artists']=df ['artists'].replace(['kaleo'], '0')

df['artists']=df ['artists'].replace(['logic'], '1')
df

,lyrics,artists
0,"""No Good"" \r Can't fight the temptation...",0
1,"""Vor Ã VaglaskÃ³gi"" \r KvÃ¶ldiÃ° er ok...",0
2,"""Glass House"" \r Well I don't need much...",0
3,"""Fool"" \r Oh yeah, come on That's right...",0
4,"""Broken Bones"" \r The devil's going to ...",0
5,"""Hunger"" \r Psychological! Damn right H...",1
6,"""Time I Give"" \r I know the time I give...",1
7,"""Logic On The Beat"" \r Yeah it's Logic ...",1
8,"""'95 Tip"" \r This right here, homie Thi...",1
9,"""On Lock"" \r Going so you know I'm tryi...",1


In [101]:
df['lyrics'] = df1['lyrics']

In [102]:
df['lyrics'][0]

'      good   \r  fight temptation vibration wo good wo good   well start run hear man come wo good wo good   mind mind hell mind   let mold break let feeling come wo good wo good   know get fever hit right know wo good say wo good   say kiss baby goodbye come love alright Heaven know wanna break apart yeah   kiss baby goodbye come love alright know try oh baby   Said good   fight temptation vibration wo good wo good tell get fever try shake try want wo good say wo good   say kiss baby goodbye come love alright Heaven know wanna break apart yeah   kiss baby goodbye come love alright know try oh yeah   yeah yeah whoa yeah baby yeah   whoa yeah want sweet little baby whoa good babe yeah say yeah day yeah baby ah yeah woo oh yeah                   '

In [103]:
#saving df
pickle.dump(df, open('df.pkl',"wb"))

In [104]:
#reading saved df
df=pd.read_pickle('df.pkl')
df

,lyrics,artists
0,good \r fight temptation vibration wo...,0
1,Vor ã vaglaskã³gi \r kvã¶ldiã ° er o...,0
2,Glass House \r need need friend bitte...,0
3,Fool \r oh yeah come right Buddy co...,0
4,Broken Bones \r devil go free man dev...,0
5,Hunger \r psychological damn right he...,1
6,time \r know time take away time live...,1
7,Logic beat \r yeah logic beat fresh t...,1
8,Tip \r right homie shit know say real...,1
9,Lock \r go know try eye prize like wa...,1


In [105]:
def clean_lyrics_from_numbers(text):
    'remove words containing numbers'
    text = re.sub('\w*\d\w*', '', text)
    return text

In [106]:
df['lyrics'] = df['lyrics'].apply(clean_lyrics_from_numbers)

In [107]:
X=df['lyrics']
y=df['artists']

In [108]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [109]:
X_train.shape,y_train.shape

((9,), (9,))

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [111]:
cv = CountVectorizer()
tf = TfidfVectorizer()

In [112]:
vectorized_lyrics = tf.fit_transform(X_train) #creating sparse matrix
vectorized_lyrics

<9x567 sparse matrix of type '<class 'numpy.float64'>'
	with 764 stored elements in Compressed Sparse Row format>

In [113]:
X_test_transf=tf.transform(X_test).toarray()


In [114]:
tf_vec=pd.DataFrame(vectorized_lyrics.todense().round(2), columns=tf.get_feature_names())  #index=X
tf_vec

,acidic,acquit,admit,affect,ah,ahead,ai,aight,alien,all,...,write,writin,written,wrong,yeah,yes,young,zone,ã¾ar,ã¾ess
0,0.00,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.06,...,0.00,0.00,0.00,0.00,0.08,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.11,0.00,0.11,0.13,0.00,0.00,0.00,...,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.06
4,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,...,0.00,0.00,0.08,0.08,0.08,0.06,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00
6,0.00,0.04,0.04,0.00,0.00,0.00,0.08,0.04,0.00,0.00,...,0.00,0.00,0.00,0.00,0.06,0.13,0.04,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00
8,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.00,...,0.00,0.06,0.00,0.00,0.16,0.04,0.05,0.06,0.00,0.00


In [115]:
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

In [116]:
def evaluations(X_test_transf,y_test, model):
    """Model evaluation metrics"""
    y_pred=model.predict(X_test_transf)

    print(f'How does model {model} score:')
    print(f'The accuracy of the model is: {round(accuracy_score(y_test, y_pred), 3)}')
    print(f'The precision of the model is: {round(precision_score(y_test, y_pred, pos_label=y),3)}')
    print(f'The recall of the model is: {round(recall_score(y_test, y_pred), 3)}')
    print(f'The f1-score of the model is: {round(f1_score(y_test, y_pred), 3)}')
    #print confusion matrix
    fig = plt.figure(figsize=(10, 8))
    cm=plot_confusion_matrix(model, X_test_transf, y_test,cmap=plt.cm.Blues, display_labels=['Kaleo','Logic'])
    print(cm)

### Naive Bayes

In [117]:
nbm = MultinomialNB(alpha=0.1)
nbm.fit(tf_vec,y_train)

MultinomialNB(alpha=0.1)

In [118]:
train_score = nbm.score(tf_vec, y_train)
print("NB train score:",train_score)

NB train score: 1.0


In [119]:
test_score=round(nbm.score(X_test_transf,y_test),1)
test_score
print("NB test score:",test_score)

NB test score: 1.0


In [120]:
cv=cross_val_score(nbm, tf_vec, y_train, cv=2, scoring='r2')
cv_mean = cv.mean()
print('Cross-validation mean:', cv.mean())

Cross-validation mean: 1.0


C:\Users\49178\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
C:\Users\49178\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [121]:
y_pred=nbm.predict(X_test_transf)
y_pred

array(['1', '1', '0'], dtype='<U1')

In [122]:
nb_ac=round(nbm.score(X_test_transf,y_test),3) 
nb_ac

1.0

In [123]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3



### Logistic Regression

In [124]:
lm=LogisticRegression()
lm.fit(tf_vec,y_train)

LogisticRegression()

In [125]:
train_score = lm.score(tf_vec, y_train)
print("NB train score:",train_score)

NB train score: 1.0


In [126]:
test_score=round(lm.score(X_test_transf,y_test),1)
test_score
print("NB test score:",test_score)

NB test score: 0.7


In [127]:
cv=cross_val_score(lm, tf_vec, y_train, cv=2, scoring='r2')
cv_mean = cv.mean()
print('Cross-validation mean:', cv.mean())

Cross-validation mean: 0.0


In [128]:
y_pred=lm.predict(X_test_transf)
y_pred

array(['1', '1', '1'], dtype=object)

In [129]:
log_ac=round(lm.score(X_test_transf,y_test),3) 
log_ac

0.667

In [130]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



C:\Users\49178\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\49178\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\49178\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random Forest

In [149]:
rfm=RandomForestClassifier()
rfm.fit(tf_vec,y_train)

RandomForestClassifier()

In [150]:
train_score = rfm.score(tf_vec, y_train)
print("NB train score:",train_score)

NB train score: 1.0


In [151]:
test_score=round(rfm.score(X_test_transf,y_test),1)
test_score
print("NB test score:",test_score)

NB test score: 0.7


In [152]:
cv=cross_val_score(rfm, tf_vec, y_train, cv=2, scoring='r2')
cv_mean = cv.mean()
print('Cross-validation mean:', cv.mean())

Cross-validation mean: -0.7499999999999998


In [153]:
y_pred=rfm.predict(X_test_transf)
y_pred

array(['0', '1', '0'], dtype=object)

In [154]:
rf_ac=round(rfm.score(X_test_transf,y_test),3) 
rf_ac

0.667

In [155]:
print(classification_report(y_test, y_pred))  #checking for 

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3



### Comparing the models

In [156]:
models = pd.DataFrame({                                              #creating all new dataframe with all the models results in
    'Models': ['Naive Bayes','Logistic Regression','Random Forest'],  #for a better comparizon
    
    'Scores': [nb_ac, log_ac,  rf_ac]
    })

models.sort_values(by='Scores', ascending=False)

,Models,Scores
0,Naive Bayes,1.000
1,Logistic Regression,0.667
2,Random Forest,0.667
